# This notebook shows how to learn weights for a logistic regression model using gradient ascent

In [1]:
#import needed libraries
import numpy as np
import graphlab as gl #The dataset is in SFrame format

## Load Amazon baby products reviews dataset

To download this dataset click [here](https://s3.amazonaws.com/static.dato.com/files/coursera/course-3/amazon_baby_subset.gl.zip)

In [2]:
%cd 'C:\Users\Rolex James\Documents\MOOCs\ML Univ of Washington\Classification\Resources\machine-learning-specialization-master\course-3'

C:\Users\Rolex James\Documents\MOOCs\ML Univ of Washington\Classification\Resources\machine-learning-specialization-master\course-3


In [3]:
#I am using a subset of the amazon baby products reviews dataset
products = gl.SFrame('amazon_baby_subset.gl/')

This non-commercial license of GraphLab Create is assigned to tolurotimibabalola@gmail.com and will expire on December 29, 2016. For commercial licensing options, visit https://dato.com/buy/.


2016-05-19 20:50:48,973 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: C:\Users\ROLEXJ~1\AppData\Local\Temp\graphlab_server_1463687448.log.0


### Exploring the dataset

In [4]:
products.head(5)

name,review,rating,sentiment
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,1
Nature's Lullabies SecondYear Sticker Calendar ...,We wanted to getsomething to keep track ...,5.0,1
Nature's Lullabies SecondYear Sticker Calendar ...,My daughter had her 1stbaby over a year ago. ...,5.0,1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4.0,1
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,Very cute interactivebook! My son loves this ...,5.0,1


In [5]:
print 'Number of positive reviews =', len(products[products['sentiment']== 1])
print 'Number of negative reviews =', len(products[products['sentiment']== -1])

Number of positive reviews = 26579
Number of negative reviews = 26493


In [6]:
print products.shape

(53072, 4)


In [7]:
#Let's look at a single review 
print products['review'][0]

All of my kids have cried non-stop when I tried to ween them off their pacifier, until I found Thumbuddy To Love's Binky Fairy Puppet.  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from it.This is a must buy book, and a great gift for expecting parents!!  You will save them soo many headaches.Thanks for this book!  You all rock!!


## Apply text cleaning to the review column

We will use  the word counts of the 193 most frequent words as the features for each review in the dataset. 
We will also remove punctuations from the 'review' column of the dataset

In [8]:
#Load the json file that contains the most important words
import json
with open('important_words.json', 'r') as f: # Reads the list of most frequent words
    important_words = json.load(f)
important_words = [str(s) for s in important_words]

In [9]:
print important_words[0:10]

['baby', 'one', 'great', 'love', 'use', 'would', 'like', 'easy', 'little', 'seat']


In [10]:
#Function to remove punctuations a string
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

In [11]:
#Create a column that contains 'clean' reviews
products['review_clean'] = products['review'].apply(remove_punctuation)

In [12]:
#Let's view the products SFrame again
products.head(5)

name,review,rating,sentiment,review_clean
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,1,All of my kids have criednonstop when I tried to ...
Nature's Lullabies SecondYear Sticker Calendar ...,We wanted to getsomething to keep track ...,5.0,1,We wanted to getsomething to keep track ...
Nature's Lullabies SecondYear Sticker Calendar ...,My daughter had her 1stbaby over a year ago. ...,5.0,1,My daughter had her 1stbaby over a year ago She ...
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4.0,1,One of babys first andfavorite books and it is ...
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,Very cute interactivebook! My son loves this ...,5.0,1,Very cute interactivebook My son loves this ...


Next we create columns that contain word counts for each word in the important_words list

In [13]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

In [14]:
#To run gradient ascent on our data we need to convert it to matrices
def get_numpy_data(data_sframe, features, label):
    data_sframe['intercept'] = 1
    features = ['intercept'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    label_sarray = data_sframe[label]
    label_array = label_sarray.to_numpy()
    return(feature_matrix, label_array)

In [15]:
#Split data into training and test sets, set seed to ensure reproducibility
train, test = products.random_split(.8, seed=1)

In [16]:
print "Number of reviews in training data: ", train.shape[0]
print "Number of reviews in test data: ", test.shape[0]

Number of reviews in training data:  42474
Number of reviews in test data:  10598


In [17]:
train_matrix, sentiment_train = get_numpy_data(train, important_words, 'sentiment')

### First we write a function to compute conditional probability with logistic link function

Recall that the logistic link function is given by:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))},
$$

where the feature vector $h(\mathbf{x}_i)$ represents the word counts of **important_words** in the review  $\mathbf{x}_i$. 

In [18]:
def predict_probability(feature_matrix, coefficients):
    '''
    produces probablistic estimate for P(y_i = +1 | x_i, w).
    estimate ranges between 0 and 1.
    '''
    score = np.dot(feature_matrix, coefficients)
    predictions = 1/(1 + np.exp(-score))
    return predictions

## Next we need to compute derivative of the log likelihood with respect to a single coefficient

This is given by:
$$
\frac{\partial\ell}{\partial w_j} = \sum_{i=1}^N h_j(\mathbf{x}_i)\left(\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})\right)
$$

This function accepts two arguments:
* `errors` vector containing $\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})$ for all $i$. This is simply the difference between the true values and our predictions using a given set of weights.
* `feature` vector containing $h_j(\mathbf{x}_i)$  for all $i$. 

In [19]:
def feature_derivative(errors, feature):     
    derivative = np.dot(errors, feature)
    return derivative

Next we compute the log likelihood for a given set of weights. This helps us to check how well the learning algorithm is doing. For each iteration the log likelihood should increase.

The log likelihood equation is given by:
$$\ell\ell(\mathbf{w}) = \sum_{i=1}^N \Big( (\mathbf{1}[y_i = +1] - 1)\mathbf{w}^T h(\mathbf{x}_i) - \ln\left(1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))\right) \Big) $$

In [20]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    logexp = np.log(1. + np.exp(-scores))
    
    # Simple check to prevent overflow
    mask = np.isinf(logexp)
    logexp[mask] = -scores[mask]
    
    lp = np.sum((indicator-1)*scores - logexp)
    return lp

## Running Gradient Ascent

The following function takes gradient steps to the optimum (the maximum point of the log likelihood function)

In [21]:
from math import sqrt

def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in xrange(max_iter):
        predictions = predict_probability(feature_matrix, coefficients)
        
        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)
        errors = indicator - predictions
        for j in xrange(len(coefficients)): # loop over each coefficient
            derivative = feature_derivative(errors, feature_matrix[:,j])
            coefficients[j] = coefficients[j] + (step_size * derivative)
            
        # Check whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print 'iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp)
                
    return coefficients

Running our logistc regression solver

In [22]:
coefficients = logistic_regression(train_matrix, sentiment_train, initial_coefficients=np.zeros(194),
                                   step_size=1e-7, max_iter=301)

iteration   0: log likelihood of observed labels = -29437.00830657
iteration   1: log likelihood of observed labels = -29433.28661525
iteration   2: log likelihood of observed labels = -29429.56826389
iteration   3: log likelihood of observed labels = -29425.85324331
iteration   4: log likelihood of observed labels = -29422.14154439
iteration   5: log likelihood of observed labels = -29418.43315805
iteration   6: log likelihood of observed labels = -29414.72807530
iteration   7: log likelihood of observed labels = -29411.02628719
iteration   8: log likelihood of observed labels = -29407.32778483
iteration   9: log likelihood of observed labels = -29403.63255941
iteration  10: log likelihood of observed labels = -29399.94060215
iteration  11: log likelihood of observed labels = -29396.25190435
iteration  12: log likelihood of observed labels = -29392.56645735
iteration  13: log likelihood of observed labels = -29388.88425256
iteration  14: log likelihood of observed labels = -29385.2052

As you can see the log likelihood increases after every iteration

## Making Predictions on the test data

In [23]:
#Convert test data to a numpy array
test_matrix, sentiment_test = get_numpy_data(test, important_words, 'sentiment')

In [24]:
#Compute probability estimates of reviews in test data being positive
pred_proba = predict_probability(test_matrix, coefficients)

In [25]:
#Predict class labels based on probability estimates
#Here I am using 0.5 as the probability threshold for predicting the positive class
#This is because logit(0) = 0.5
pred_labels = np.where(pred_proba >= 0.5, +1, -1)

In [26]:
#View the first 10 predictions
pred_labels[0:10]

array([-1, -1,  1, -1,  1, -1,  1,  1,  1, -1])

Alternatively we can use the scores (i.e. dot product of the coefficients and feature values) for predicting class labels

In [27]:
pred_scores = np.dot(test_matrix, coefficients)
pred_labels_scores = np.where(pred_scores > 0, +1, -1)
pred_labels_scores[0:10]

array([-1, -1,  1, -1,  1, -1,  1,  1,  1, -1])

## Measuring accuracy on test data

$$
\mbox{accuracy} = \frac{\mbox{# correctly classified data points}}{\mbox{# total data points}}
$$

In [28]:
num_mistakes = len(test) - np.sum(pred_labels == np.array(test['sentiment']))
accuracy = np.sum(pred_labels == np.array(test['sentiment'])) / float(len(test))
print "-----------------------------------------------------"
print 'Number of test reviews correctly classified =', len(test) - num_mistakes
print 'Number of test reviews incorrectly classified =', num_mistakes
print 'Total number of reviews                       =', len(test)
print "-----------------------------------------------------"
print 'Accuracy = %.2f' % accuracy

-----------------------------------------------------
Number of test reviews correctly classified = 7875
Number of test reviews incorrectly classified = 2723
Total number of reviews                       = 10598
-----------------------------------------------------
Accuracy = 0.74


This model does pretty well on the test set. It is better than a majority classifier which would have an accuaracy of about 50%

## Finally, let's see the words most associated with positive & negative sentiment

In [29]:
coefficients = list(coefficients[1:]) # exclude intercept

In [30]:
#Create a tuple that contains a word and it's corresponding coefficient
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(important_words, coefficients)]
#Sort the tuple in descending order
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)

The 10 most positive words

In [31]:
word_coefficient_tuples[:10]

[('love', 0.053359539345264127),
 ('great', 0.052190691148159253),
 ('easy', 0.051963344707537398),
 ('loves', 0.036000365451516796),
 ('little', 0.035950203379285416),
 ('well', 0.0241904367857248),
 ('perfect', 0.023882633500110551),
 ('old', 0.015398688484057191),
 ('nice', 0.015388471138522141),
 ('daughter', 0.014054417887905957)]

The 10 most negative words

In [32]:
word_coefficient_tuples[-10:]

[('monitor', -0.021576851648122553),
 ('return', -0.021908406431109355),
 ('back', -0.022232458157612718),
 ('get', -0.023441036058314144),
 ('disappointed', -0.023618461190266123),
 ('even', -0.025038615750094854),
 ('work', -0.027567816668181165),
 ('money', -0.031273253813973237),
 ('product', -0.035365453358602431),
 ('would', -0.044427924435117498)]